In [1]:
import json

In [2]:
relative_path= "../../data/in/"

In [3]:
import glob
file_paths= glob.glob("../../data/in/*.json")

In [18]:
def check_number_of_countries(file_paths):
    previous= 0
    for index, path in enumerate(file_paths):
        
        with open(path, 'r') as file:
            current= json.load(file)
        
        print(index, path[-15:-5], len(current), len(current) - previous)
        previous= len(current)

In [ ]:
check_number_of_countries(file_paths)

In [33]:
def check_which_countries_are_missing(file_paths):
    with open(file_paths[-1], 'r') as file:
        final_date= json.load(file)
    
    missing= []
    
    for index, path in enumerate(file_paths[:-1]):
        with open(path, 'r') as file:
            current= json.load(file)
        
        for country in current.keys():
            if country not in final_date.keys():
                missing.append(country)
    
    return set(missing)    

In [34]:
missing_countries= check_which_countries_are_missing(file_paths)
missing_countries

{'Belgium', 'Bulgaria', 'Japan', 'Poland', 'Slovenia'}

In [75]:
def linear_interpolation_for_China(china_dict):
    d_cases= 88
    d_deaths= 3.5
        
    for index in range(len(china_dict['cases'][:9])): 
        china_dict['cases'][index]= int(index*d_cases)
    
    for index in range(len(china_dict['deaths'][:10])):
        china_dict['deaths'][index]= int(index*d_deaths)
        
    # No need to do interpolation for recoveries since they're all zero 

In [76]:
def generate_final_country_lists(file_paths, missing):
    with open(file_paths[-1], 'r') as file:
        final_date= json.load(file)
    
    final_country_dict= {}
        
    for country in missing:
        final_country_dict[country]= {'cases': [], 'deaths': [], 'recoveries': []}
    
    for country in final_date.keys():
        final_country_dict[country]= {'cases': [], 'deaths': [], 'recoveries': []}    
    
    for index, path in enumerate(file_paths):
        with open(path, 'r') as file:
            current= json.load(file)
        
        for country in final_country_dict:
            if country not in current:
                try:
                    final_country_dict[country]['cases'].append(final_country_dict[country]['cases'][-1])
                    final_country_dict[country]['deaths'].append(final_country_dict[country]['deaths'][-1])
                    final_country_dict[country]['recoveries'].append(final_country_dict[country]['recoveries'][-1])
                except:
                    final_country_dict[country]['cases'].append(0)
                    final_country_dict[country]['deaths'].append(0)
                    final_country_dict[country]['recoveries'].append(0)
            else:
                final_country_dict[country]['cases'].append(current[country]['cases'])
                final_country_dict[country]['deaths'].append(current[country]['deaths'])
                final_country_dict[country]['recoveries'].append(current[country]['recoveries'])
        
    linear_interpolation_for_China(final_country_dict['China'])
    
    
    with open(relative_path + 'all_countries_data.json', 'w') as file:
        json.dump(final_country_dict, file)
    
#     for country in final_country_dict:
#         for key in final_country_dict[country]:
#             print(country, key, final_country_dict[country][key])
#         print()

In [77]:
generate_final_country_lists(file_paths, missing_countries)